In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('kmeans_binary_labeled_helpfulness_data_with_modified_review.csv')

In [3]:
df.head()

,Unnamed: 0,apartmentName,url,rating,review,zip,propertyManager,minRent,maxRent,minSqft,maxSqft,avgRating,helpfulness,days_since_review,state,city,calculated_score,kmeans_label,modified_review
0,0,Union 505,https://www.apartments.com/union-505-albuquerq...,5.0,"A very welcoming community, convenient locatio...",NM 87102,https://www.apartments.com/pmc/greystar/dv0gtnl/,850,2662,324.0,755.0,4.0,0,67.0,NM,ALBUQUERQUE,0.0,0,State NM and City ALBUQUERQUE. A very welcomi...
1,1,Union 505,https://www.apartments.com/union-505-albuquerq...,3.0,Union 505 is in a great location for Albuquerq...,NM 87102,https://www.apartments.com/pmc/greystar/dv0gtnl/,850,2662,324.0,755.0,4.0,0,71.0,NM,ALBUQUERQUE,0.0,0,State NM and City ALBUQUERQUE. Union 505 is i...
2,2,Union 505,https://www.apartments.com/union-505-albuquerq...,4.0,My Experience here at union505 is good only th...,NM 87102,https://www.apartments.com/pmc/greystar/dv0gtnl/,850,2662,324.0,755.0,4.0,2,124.0,NM,ALBUQUERQUE,1.0,1,State NM and City ALBUQUERQUE. My Experience ...
3,3,Union 505,https://www.apartments.com/union-505-albuquerq...,4.0,I love the apartment. Its very cozy and straig...,NM 87102,https://www.apartments.com/pmc/greystar/dv0gtnl/,850,2662,324.0,755.0,4.0,2,129.0,NM,ALBUQUERQUE,1.0,1,State NM and City ALBUQUERQUE. I love the apa...
4,4,Union 505,https://www.apartments.com/union-505-albuquerq...,4.0,The apartments are very spacious. The leasing ...,NM 87102,https://www.apartments.com/pmc/greystar/dv0gtnl/,850,2662,324.0,755.0,4.0,2,132.0,NM,ALBUQUERQUE,1.0,1,State NM and City ALBUQUERQUE. The apartments...


In [4]:
df.drop(columns = ['Unnamed: 0'], inplace = True)

In [5]:
df['review_word_count'] = df['review'].str.split().str.len()

In [6]:
from sklearn.model_selection import train_test_split

In [7]:
train_x, test_x, train_labels, test_labels = train_test_split(df['review_word_count'], df['kmeans_label'], test_size=0.2, random_state=42)

In [8]:
max_word_count = train_x.max()

In [9]:
min_word_count = train_x.min()

In [10]:
print(max_word_count)
print(min_word_count)

1097
1


In [11]:
quantile25 = train_x.quantile(0.25)

In [12]:
quantile50 = train_x.quantile(0.5)

In [13]:
quantile75 = train_x.quantile(0.75)

In [14]:
count_list = [0, 0, 0, 0, 0, 0, 0, 0]

for i in (list(train_x.index)):
    if(train_x.at[i] <= quantile25):
        count_list[0 + int(train_labels.at[i])] += 1
    elif(train_x.at[i] <= quantile50):
        count_list[2 + int(train_labels.at[i])] += 1
    elif(train_x.at[i] <= quantile75):
        count_list[4 + int(train_labels.at[i])] += 1
    else:
        count_list[6 + int(train_labels.at[i])] += 1

print(count_list)   
        

[5424, 2814, 4570, 3659, 3718, 4294, 2758, 5329]


In [15]:
quantile_list = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
count_0 = 0
count_1 = 0
for i in range(len(quantile_list)):
    quantile = train_x.quantile(quantile_list[i])
    count_0 = 0
    count_1 = 0
    misclassified = 0
    for j in (list(train_x.index)):
        if(train_x.at[j] <= quantile):
            if(int(train_labels.at[j]) == 0):
                count_0 += 1
            else:
                misclassified += 1
        else:
            if(int(train_labels.at[j]) == 1):
                count_1 += 1
            else:
                misclassified += 1

    print('For Quantile ', quantile_list[i], ' - 0 : ', count_0, '  1: ',count_1, ' Misclassified : ', misclassified)


For Quantile  0.1  - 0 :  2305   1:  15012  Misclassified :  15249
For Quantile  0.2  - 0 :  4401   1:  13852  Misclassified :  14313
For Quantile  0.3  - 0 :  6392   1:  12698  Misclassified :  13476
For Quantile  0.4  - 0 :  8252   1:  11205  Misclassified :  13109
For Quantile  0.5  - 0 :  9994   1:  9623  Misclassified :  12949
For Quantile  0.6  - 0 :  11614   1:  8050  Misclassified :  12902
For Quantile  0.7  - 0 :  13060   1:  6300  Misclassified :  13206
For Quantile  0.8  - 0 :  14309   1:  4312  Misclassified :  13945
For Quantile  0.9  - 0 :  15456   1:  2243  Misclassified :  14867


In [16]:
quantile_length = train_x.quantile(0.6)

from sklearn.metrics import classification_report

true_labels = []
predicted_labels = []

for i in (list(test_x.index)):
    true_labels.append(int(test_labels.at[i]))
    if(test_x.at[i] <= quantile_length):
        predicted_labels.append(0)
    else:
        predicted_labels.append(1)


In [17]:
report = classification_report(true_labels, predicted_labels)
print(report)

              precision    recall  f1-score   support

           0       0.59      0.71      0.64      4139
           1       0.62      0.50      0.55      4003

    accuracy                           0.60      8142
   macro avg       0.61      0.60      0.60      8142
weighted avg       0.61      0.60      0.60      8142



In [22]:
index_values = test_x.index.values.tolist()
final_df = pd.DataFrame()
misclassified_rows = []
for i in range(len(index_values)):
    if(true_labels[i] != predicted_labels[i]):
        misclassified_rows.append(index_values[i])

misclassified_df = df.loc[misclassified_rows]

In [23]:
len(misclassified_df)

3238

In [24]:
misclassified_df.to_csv('Rule_based_misclassified_rows.csv')

In [26]:
misclassified_df['review'].head()

5712     Been living here for four years and it’s been ...
6946     Midtown Towers has recently gone under a chang...
40426    As a first time renter, I've found this apartm...
4859     I have lived @ The Marq for over a year and co...
4009     If you want a nice quiet place to reside this ...
Name: review, dtype: object